In [72]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import glob
import pandas as pd
from scipy.io import arff
import logging
import os
import pathlib

# Set up logger
logger = logging.getLogger(__name__)

In [73]:
SCRIPT_DIR = pathlib.Path(os.getcwd()).absolute()
DATA_DIR = os.path.join(SCRIPT_DIR.parent, "data")
PREPROCESSED_DATA_DIR = f'{DATA_DIR}/1_preprocessed/'

In [74]:
def identify_categorical_and_numerical(df, unique_threshold=0.05):
    """
    Identifies categorical and numerical columns in a DataFrame.

    Args:
        df: The pandas DataFrame.
        unique_threshold: The threshold for the proportion of unique values 
                           to consider a column categorical.

    Returns:
        A tuple containing two lists: categorical columns and numerical columns.
    """
    categorical_cols = []
    numerical_cols = []

    for col in df.columns:
        if df[col].dtype == 'object' or df[col].nunique() / len(df) <= unique_threshold:
            categorical_cols.append(col)
        else:
            numerical_cols.append(col)

    return categorical_cols, numerical_cols

In [75]:
def load_datasets(file_pattern: str) -> list[pd.DataFrame]:
    files = glob.glob(file_pattern)

    # List to store the dataframes
    dfs = []

    # Loop through each matching file
    for file in files:
        # Load the ARFF file
        raw_data, meta = arff.loadarff(file)

        # Log the file being loaded
        logger.debug(f"Loading {file}")

        # Convert the ARFF data to a pandas DataFrame
        df = pd.DataFrame(raw_data, columns=meta.names())

        # Decode byte-strings if necessary
        df = df.map(lambda x: x.decode() if isinstance(x, bytes) else x)

        # Append dataframe to the list
        dfs.append(df)

    return dfs

In [76]:
def save_to_csv(df, filename):
    """Saves the DataFrame to a CSV file.

    Args:
      df: The pandas DataFrame to save.
      filename: The name of the CSV file (e.g., 'preprocessed_data.csv').
    """
    try:
        df.to_csv(f'{PREPROCESSED_DATA_DIR}/{filename}', index=False)  # Set index=False to avoid saving the index
        print(f"DataFrame saved to {filename}")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")

In [77]:
def preprocess_dataset(df):
    # Replace ? with nan for correct imputation
    df.replace("?", np.nan, inplace=True)

    class_col_name = df.columns[-1]
    class_col = df[class_col_name]    
    df.drop(class_col_name, axis=1, inplace=True)


    # Get categorical and numerical columns based on dtype and heuristics
    categorical_cols, numeric_cols = identify_categorical_and_numerical(df)
    print('AFTER METHOD')
    print('categorical columns', categorical_cols)
    print('numeric columns', numeric_cols)

    # Create a column transformer for preprocessing
    preprocessor = ColumnTransformer(
        transformers=[
            (
                "num",
                Pipeline(
                    steps=[
                        ("imputer", SimpleImputer(strategy="mean")),  # Fill missing with mean
                        ("scaler", MinMaxScaler()),  # Min-Max scaling
                    ]
                ),
                numeric_cols,
            ),
            (
                "cat",
                Pipeline(
                    steps=[
                        (
                            "imputer",
                            SimpleImputer(strategy="most_frequent"),
                        ),  # Fill missing with mode
                        # Use 'passthrough' to handle categorical data
                        ("passthrough", "passthrough"),
                    ]
                ),
                categorical_cols,
            ),
        ],
    )

    # Apply the preprocessor to the dataframe, excluding ignored columns
    processed_array = preprocessor.fit_transform(df)

    # Separate binary and non-binary categorical columns
    binary_cols = [col for col in categorical_cols if df[col].nunique() <= 2]
    non_binary_cols = [col for col in categorical_cols if df[col].nunique() > 2]

    print('binary cols', binary_cols)
    print('non binary cols', non_binary_cols)
    
    # Extract processed numerical data
    processed_numeric_df = pd.DataFrame(processed_array[:, :len(numeric_cols)], columns=numeric_cols)
    processed_categorical_df = pd.DataFrame(processed_array[:, len(numeric_cols):], columns=categorical_cols)

    encoded_categorical_binary_df = processed_categorical_df.drop(non_binary_cols, axis=1)

    # Label Encoder
    for col in binary_cols:
        le = LabelEncoder()
        encoded_categorical_binary_df[col] = le.fit_transform(encoded_categorical_binary_df[col])
   

    # One-hot encode categorical features (the literature suggests OneHotEcoding for k-means clustering approaches )
    ohe = OneHotEncoder(sparse_output=False)
    encoded_categorical_non_binary_array = ohe.fit_transform(processed_categorical_df[non_binary_cols])  # Encode categorical part

    # Rename encoded columns
    encoded_columns = []
    for i, cat_col in enumerate(non_binary_cols):
        for j in range(ohe.categories_[i].size):
            encoded_columns.append(f"{cat_col}_{ohe.categories_[i][j]}")

    # print(encoded_columns)
    encoded_categorical_non_binary_df = pd.DataFrame(encoded_categorical_non_binary_array, columns=encoded_columns)

    # Combine DataFrames
    final_df = pd.concat([processed_numeric_df, encoded_categorical_binary_df, encoded_categorical_non_binary_df], axis=1)

    # Encoding the class column (for metrics)
    le = LabelEncoder()
    final_df['class'] = le.fit_transform(class_col)

    return final_df

In [78]:

DATA_DIR

'/home/cajifan/Documents/MAI/1S/IML/mai-intro-to-machine-learning/work3/data'

In [79]:
data_path = f"{DATA_DIR}/0_raw/hepatitis.arff"
dfs = load_datasets(data_path)

df = dfs[0]

preprocessed_df = preprocess_dataset(df)
save_to_csv(preprocessed_df, 'hepatitis.csv')
preprocessed_df

AFTER METHOD
categorical columns ['SEX', 'STEROID', 'ANTIVIRALS', 'FATIGUE', 'MALAISE', 'ANOREXIA', 'LIVER_BIG', 'LIVER_FIRM', 'SPLEEN_PALPABLE', 'SPIDERS', 'ASCITES', 'VARICES', 'HISTOLOGY']
numeric columns ['AGE', 'BILIRUBIN', 'ALK_PHOSPHATE', 'SGOT', 'ALBUMIN', 'PROTIME']
binary cols ['SEX', 'STEROID', 'ANTIVIRALS', 'FATIGUE', 'MALAISE', 'ANOREXIA', 'LIVER_BIG', 'LIVER_FIRM', 'SPLEEN_PALPABLE', 'SPIDERS', 'ASCITES', 'VARICES', 'HISTOLOGY']
non binary cols []
DataFrame saved to hepatitis.csv


AGE BILIRUBIN ALK_PHOSPHATE      SGOT   ALBUMIN   PROTIME  SEX  \
0    0.323944  0.090909      0.219331  0.006309   0.44186  0.618523    1   
1    0.605634  0.077922      0.405204  0.044164  0.325581  0.618523    0   
2         1.0  0.051948      0.260223  0.028391   0.44186  0.618523    0   
3    0.338028  0.051948      0.074349  0.059937   0.44186       0.8    0   
4    0.380282  0.090909       0.29489  0.293375   0.44186  0.618523    0   
..        ...       ...           ...       ...       ...       ...  ...   
150  0.549296  0.948052       0.29489  0.359621   0.27907       0.5    0   
151  0.521127  0.077922      0.371747  0.201893  0.511628  0.618523    0   
152  0.760563  0.064935      0.182156  0.009464  0.465116  0.618523    0   
153  0.647887  0.155844      0.204461  0.007886  0.465116      0.48    1   
154  0.507042  0.116883      0.275093  0.007886  0.232558      0.42    0   

     STEROID  ANTIVIRALS  FATIGUE  MALAISE  ANOREXIA  LIVER_BIG  LIVER_FIRM  \
0          0           0        0        0         0          0           0   
1          0           0        1        0         0          0           0   
2          1           0        1        0         0          1           0   
3          1           1        0        0         0          1           0   
4          1           0        0        0         0          1           0   
..       ...         ...      ...      ...       ...        ...         ...   
150        1           0        1        1         1          1           0   
151        1           0        1        0         0          1           1   
152        0           0        1        1         0          0           1   
153        0           0        1        0         0          1           0   
154        1           0        1        0         0          1           0   

     SPLEEN_PALPABLE  SPIDERS  ASCITES  VARICES  HISTOLOGY  class  
0                  0        0        0        0          0      1  
1                  0        0        0        0          0      1  
2                  0        0        0        0          0      1  
3                  0        0        0        0          0      1  
4                  0        0        0        0          0      1  
..               ...      ...      ...      ...        ...    ...  
150                0        1        1        1          1      0  
151                0        0        0        0          1      1  
152                0        1        0        0          1      1  
153                1        1        0        1          1      1  
154                1        1        1        0          1      0  

[155 rows x 20 columns]

In [80]:
data_path = f"{DATA_DIR}/0_raw/vowel.arff"
dfs = load_datasets(data_path)
df = dfs[0]

preprocessed_df = preprocess_dataset(df)
save_to_csv(preprocessed_df, f'vowel.csv')
preprocessed_df

AFTER METHOD
categorical columns ['Train_or_Test', 'Speaker_Number', 'Sex']
numeric columns ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7', 'Feature_8', 'Feature_9']
binary cols ['Train_or_Test', 'Sex']
non binary cols ['Speaker_Number']
DataFrame saved to vowel.csv


Feature_0 Feature_1 Feature_2 Feature_3 Feature_4 Feature_5 Feature_6  \
0     0.36815  0.266541  0.463757   0.84205  0.494947  0.778691  0.390816   
1    0.441218  0.278828  0.457631  0.732699   0.47044  0.875435   0.39932   
2    0.723888  0.341525  0.232517  0.410988  0.358767  0.757193   0.32585   
3    0.684778  0.485665  0.252425  0.639461  0.271349  0.599431  0.329932   
4    0.611944  0.505986  0.418836  0.652668  0.124811  0.505849  0.656803   
..        ...       ...       ...       ...       ...       ...       ...   
985  0.461827  0.686358  0.270546  0.318806  0.466145  0.713563  0.718707   
986  0.341452  0.768589  0.405309  0.174855  0.436079  0.822321  0.733673   
987   0.28829  0.588059  0.652118  0.378236  0.597271   0.58963  0.458503   
988   0.22178  0.661468  0.651353  0.581881  0.618999  0.427126  0.086735   
989  0.449649  0.566793   0.46146  0.447438  0.648813  0.440405  0.445578   

    Feature_7 Feature_8 Feature_9  ...  Speaker_Number_Mike  \
0    0.546819  0.252909  0.281534  ...                  0.0   
1    0.541116  0.339493  0.387516  ...                  0.0   
2    0.590936  0.275154  0.530234  ...                  0.0   
3    0.458583  0.520876  0.287711  ...                  0.0   
4    0.343037  0.646817  0.417425  ...                  0.0   
..        ...       ...       ...  ...                  ...   
985  0.408463   0.23922  0.593953  ...                  0.0   
986  0.614346  0.266256  0.448309  ...                  0.0   
987  0.544418  0.287474  0.383615  ...                  0.0   
988  0.346038  0.256674  0.419376  ...                  0.0   
989  0.422569  0.194045  0.772757  ...                  0.0   

     Speaker_Number_Nick  Speaker_Number_Penny  Speaker_Number_Rich  \
0                    0.0                   0.0                  0.0   
1                    0.0                   0.0                  0.0   
2                    0.0                   0.0                  0.0   
3                    0.0                   0.0                  0.0   
4                    0.0                   0.0                  0.0   
..                   ...                   ...                  ...   
985                  0.0                   0.0                  0.0   
986                  0.0                   0.0                  0.0   
987                  0.0                   0.0                  0.0   
988                  0.0                   0.0                  0.0   
989                  0.0                   0.0                  0.0   

     Speaker_Number_Rose  Speaker_Number_Sarah  Speaker_Number_Sue  \
0                    0.0                   0.0                 0.0   
1                    0.0                   0.0                 0.0   
2                    0.0                   0.0                 0.0   
3                    0.0                   0.0                 0.0   
4                    0.0                   0.0                 0.0   
..                   ...                   ...                 ...   
985                  0.0                   0.0                 0.0   
986                  0.0                   0.0                 0.0   
987                  0.0                   0.0                 0.0   
988                  0.0                   0.0                 0.0   
989                  0.0                   0.0                 0.0   

     Speaker_Number_Tim  Speaker_Number_Wendy  class  
0                   0.0                   0.0      8  
1                   0.0                   0.0      2  
2                   0.0                   0.0      1  
3                   0.0                   0.0      0  
4                   0.0                   0.0      5  
..                  ...                   ...    ...  
985                 0.0                   1.0      3  
986                 0.0                   1.0      9  
987                 0.0                   1.0      4  
988                 0.0                   1.0     10  
989                 0.0

In [81]:
data_path = f"{DATA_DIR}/0_raw/mushroom.arff"
dfs = load_datasets(data_path)
df = dfs[0]

preprocessed_df = preprocess_dataset(df)
save_to_csv(preprocessed_df, 'mushroom.csv')
preprocessed_df

AFTER METHOD
categorical columns ['cap-shape', 'cap-surface', 'cap-color', 'bruises?', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']
numeric columns []
binary cols ['bruises?', 'gill-attachment', 'gill-spacing', 'gill-size', 'stalk-shape', 'veil-type']
non binary cols ['cap-shape', 'cap-surface', 'cap-color', 'odor', 'gill-color', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']
DataFrame saved to mushroom.csv


bruises?  gill-attachment  gill-spacing  gill-size  stalk-shape  \
0            1                1             0          0            0   
1            0                1             0          1            1   
2            1                1             0          0            0   
3            1                1             0          1            0   
4            0                1             0          1            1   
...        ...              ...           ...        ...          ...   
8119         0                1             1          0            1   
8120         1                1             0          0            1   
8121         0                1             1          0            1   
8122         0                1             0          1            1   
8123         0                1             0          1            1   

      veil-type  cap-shape_b  cap-shape_c  cap-shape_f  cap-shape_k  ...  \
0             0          0.0          0.0          0.0          0.0  ...   
1             0          0.0          0.0          0.0          1.0  ...   
2             0          0.0          0.0          0.0          0.0  ...   
3             0          0.0          0.0          0.0          0.0  ...   
4             0          0.0          0.0          0.0          0.0  ...   
...         ...          ...          ...          ...          ...  ...   
8119          0          0.0          0.0          1.0          0.0  ...   
8120          0          0.0          0.0          1.0          0.0  ...   
8121          0          0.0          0.0          0.0          0.0  ...   
8122          0          0.0          0.0          1.0          0.0  ...   
8123          0          0.0          0.0          0.0          1.0  ...   

      population_v  population_y  habitat_d  habitat_g  habitat_l  habitat_m  \
0              0.0           0.0        0.0        0.0        0.0        1.0   
1              1.0           0.0        1.0        0.0        0.0        0.0   
2              0.0           0.0        0.0        0.0        0.0        1.0   
3              1.0           0.0        0.0        0.0        0.0        0.0   
4              1.0           0.0        0.0        0.0        1.0        0.0   
...            ...           ...        ...        ...        ...        ...   
8119           0.0           0.0        0.0        1.0        0.0        0.0   
8120           0.0           1.0        1.0        0.0        0.0        0.0   
8121           0.0           0.0        0.0        1.0        0.0        0.0   
8122           1.0           0.0        1.0        0.0        0.0        0.0   
8123           1.0           0.0        1.0        0.0        0.0        0.0   

      habitat_p  habitat_u  habitat_w  class  
0           0.0        0.0        0.0      0  
1           0.0        0.0        0.0      1  
2           0.0        0.0        0.0      0  
3           0.0        1.0        0.0      1  
4           0.0        0.0        0.0      1  
...         ...        ...        ...    ...  
8119        0.0        0.0        0.0      0  
8120        0.0        0.0        0.0      0  
8121        0.0        0.0        0.0      0  
8122        0.0        0.0        0.0      1  
8123        0.0        0.0        0.0      1  

[8124 rows x 112 columns]

In [82]:
data_path = f"{DATA_DIR}/1_preprocessed/hepatitis.csv"
df  = pd.read_csv(data_path)
df

AGE  BILIRUBIN  ALK_PHOSPHATE      SGOT   ALBUMIN   PROTIME  SEX  \
0    0.323944   0.090909       0.219331  0.006309  0.441860  0.618523    1   
1    0.605634   0.077922       0.405204  0.044164  0.325581  0.618523    0   
2    1.000000   0.051948       0.260223  0.028391  0.441860  0.618523    0   
3    0.338028   0.051948       0.074349  0.059937  0.441860  0.800000    0   
4    0.380282   0.090909       0.294890  0.293375  0.441860  0.618523    0   
..        ...        ...            ...       ...       ...       ...  ...   
150  0.549296   0.948052       0.294890  0.359621  0.279070  0.500000    0   
151  0.521127   0.077922       0.371747  0.201893  0.511628  0.618523    0   
152  0.760563   0.064935       0.182156  0.009464  0.465116  0.618523    0   
153  0.647887   0.155844       0.204461  0.007886  0.465116  0.480000    1   
154  0.507042   0.116883       0.275093  0.007886  0.232558  0.420000    0   

     STEROID  ANTIVIRALS  FATIGUE  MALAISE  ANOREXIA  LIVER_BIG  LIVER_FIRM  \
0          0           0        0        0         0          0           0   
1          0           0        1        0         0          0           0   
2          1           0        1        0         0          1           0   
3          1           1        0        0         0          1           0   
4          1           0        0        0         0          1           0   
..       ...         ...      ...      ...       ...        ...         ...   
150        1           0        1        1         1          1           0   
151        1           0        1        0         0          1           1   
152        0           0        1        1         0          0           1   
153        0           0        1        0         0          1           0   
154        1           0        1        0         0          1           0   

     SPLEEN_PALPABLE  SPIDERS  ASCITES  VARICES  HISTOLOGY  class  
0                  0        0        0        0          0      1  
1                  0        0        0        0          0      1  
2                  0        0        0        0          0      1  
3                  0        0        0        0          0      1  
4                  0        0        0        0          0      1  
..               ...      ...      ...      ...        ...    ...  
150                0        1        1        1          1      0  
151                0        0        0        0          1      1  
152                0        1        0        0          1      1  
153                1        1        0        1          1      1  
154                1        1        1        0          1      0  

[155 rows x 20 columns]